# Generate Passwords for Cracking Lab
In this lab, participants will attempt to crack password in several hash formats:
- MS Word
- Zip
- PDF
- NTLM (Active Directory)

The goal is to crack passwords for more critical documents or accounts. The more critical the document or account, the more points that are awarded for cracking it. Some accounts/documents will be difficult to crack without planning or research: brute force will not work.

## Documents

Assets:
- A directory with 100 word documents, 100 zip files, 100 PDFs
- A file with 5000 Active Directory accounts, hashes, and role
- 10% critical
-- specific assets
-- domain admins
- 15% high value
-- executives
-- service accounts
- 25% medium value
-- trading floor
-- senior analysts
-- IT and developers
- 50% low value
-- everyone else

NTLM Passwords:
- impossible : random >= 15
- hard
    random >=12
    2 words + rules
- password
    from my list + rules
- company custom
    from custom list
- complex
    words + rules
- simple : ?u?l?l?l?d?d?s


## Random Passwords
Depending on the length, these can be hard to crack. Humans tend not to use long random passwords, but we should include some.

## Simple Passwords

# .\hashcat.exe --force --stdout --custom-charset1='2' --custom-charset2='0' --custom-charset3='012' -a 6 ..\words\dictionary\english.txt '?1?2?3?d?s' > ..\words\datewords.txt 
# .\hashcat.exe --force --stdout -a 0 -j 'c' ..\words\datewords.txt > simplewords.txt


## Based on Password
I have pregenerated a file that contains every variant of the word password. These are all based on character subsitutions and are all 8 characters long. However, there are other shorted common lists. To make our list we want to include some of these straight but also want to run them through some rules.
.\hashcat.exe --force --stdout -a 0 -r rules\\best64.rule ..\words\based\password.txt > password.txt

In [27]:
import random
import csv

In [36]:
# Number of accounts to generate
accounts_to_generate = 5000
# A list of accounts, will contain {'username', 'department', 'criticality', 'password', 'hash'}
accounts = []
# Generate usernames
fd = open('D:\\Projects\\Password Cracking Lab\\words\\first_names', 'r')
first_names = list(map(str.strip, fd.readlines()))
fd.close()

fd = open('D:\\Projects\\Password Cracking Lab\\words\\last_names', 'r')
last_names = list(map(str.strip, fd.readlines()))
fd.close()

fd = open('D:\\Projects\\Password Cracking Lab\\departments.csv', 'r')
departments = list(csv.DictReader(fd))
fd.close()

In [37]:
# explore loaded data
# print(departments)

In [39]:
for i in range(accounts_to_generate):
    account_name = random.choice(first_names) + '.' + random.choice(last_names)
    dept = random.choice(departments)
    accounts.append({'account_name': account_name, 
                   'department': dept['department'], 
                   'criticality': dept['criticality']
                   })

In [40]:
accounts

[{'account_name': 'ludivina.donley',
  'department': 'Derivatives Trading',
  'criticality': 'medium'},
 {'account_name': 'jodie.poisel',
  'department': 'Executive Office',
  'criticality': 'high'},
 {'account_name': 'serene.lenny',
  'department': 'Wealth Management',
  'criticality': 'medium'},
 {'account_name': 'mignon.fiorilli',
  'department': 'Securities',
  'criticality': 'low'},
 {'account_name': 'roobbie.escarsega',
  'department': 'Trading Floor',
  'criticality': 'high'},
 {'account_name': 'herman.newnham',
  'department': 'Software Development',
  'criticality': 'medium'},
 {'account_name': 'gerhardine.huffner',
  'department': 'Small Business Banking',
  'criticality': 'low'},
 {'account_name': 'mellicent.reader',
  'department': 'Research',
  'criticality': 'medium'},
 {'account_name': 'oliver.kalupa',
  'department': 'Clerical and IT',
  'criticality': 'high'},
 {'account_name': 'josephus.urias',
  'department': 'Executive Office',
  'criticality': 'high'},
 {'account_n